In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from io import BytesIO

In [ ]:
from dmp.dmp import DMPs_cartesian
from dmp.obstacle_superquadric import Obstacle_Dynamic as sq_dyn

In [ ]:
# Trajectory 1
# goal_move_to_peg = np.array([0.488574563751,
#                             -0.184923096437,
#                             0.262908357465])

# goal_move_to_ring = np.array([0.505577739181,
#                               0.310174815877,
#                               0.045214734967])

# x0_move_to_peg = goal_move_to_ring

# x0_move_to_ring = np.array([0.307, 0, 0.483])



In [ ]:
# #DMP Ini
# K=1050.0
# alpha_s = 4.0
# tol=0.01

# MP = DMPs_cartesian(K=K, alpha_s=alpha_s, tol=tol)

In [ ]:
# #DMP evolution
# MP.x_0=x0_move_to_ring
# MP.x_goal=goal_move_to_ring
# MP.reset_state()
# x_track_static_toring = np.array([MP.x_0])
# flag_conv = False
# while not flag_conv:
#     MP.step()
#     x_track_static_toring = np.append(x_track_static_toring, np.array([MP.x]), axis=0)
#     flag_conv = np.linalg.norm(MP.x - MP.x_goal) < MP.tol

In [ ]:
# #DMP 3D plot
# fig=plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot(x_track_static_toring[:, 0], x_track_static_toring[:, 1], x_track_static_toring[:, 2],'-k', linewidth=1.5)

In [ ]:
# Trajectory 2
# Trajectory
t_des = np.linspace(0.0, 1.0, 1000)
gamma = np.transpose(
    [np.cos(np.pi * t_des),
    np.sin(np.pi * t_des),
    t_des * 0.5])

gamma2 = np.transpose(
    [np.cos(np.pi * t_des),
    1-np.sin(np.pi * t_des),
    t_des * 0.5])


In [ ]:
#DMP Ini
K=1050.0
alpha_s = 4.0
tol=0.01
dt=0.01

MP = DMPs_cartesian(K=K, alpha_s=alpha_s, tol=tol)
MP2= DMPs_cartesian(K=K, alpha_s=alpha_s, tol=tol)

In [ ]:
print(gamma2)

In [ ]:
#DMP try
MP.imitate_path(t_des=t_des, x_des=gamma)
MP2.imitate_path(t_des=t_des, x_des=gamma2)

gamma_, _, _, _ = MP.rollout()
gamma2_, _, _, _ = MP2.rollout()
t_des_ = np.linspace(0.0, 1.0, np.shape(gamma)[0])

In [ ]:
# Plot Ellipsoid
# Define the ellipsoid parameters
a = 0.2  # Semi-axis length in the x-direction
b = 0.1  # Semi-axis length in the y-direction
c = 0.15  # Semi-axis length in the z-direction

# Create a grid of values for theta and phi
theta = np.linspace(0, 2 * np.pi, 20)
phi = np.linspace(0, np.pi, 10)
theta, phi = np.meshgrid(theta, phi)

# Parametric equations for the ellipsoid
x = a * np.sin(phi) * np.cos(theta) + 0.0
y = b * np.sin(phi) * np.sin(theta) + 1.0
z = c * np.cos(phi) + 0.25

In [ ]:
MP.x_0 = gamma_[0]
MP.x_goal = gamma_[-1]

MP2.x_0 = gamma2_[0]
MP2.x_goal = gamma2_[-1]

MP.reset_state()
MP2.reset_state()
gamma_tra = np.array([MP.x_0])
gamma2_tra= np.array([MP2.x_0])

In [ ]:
dyn_lambda = 10.0
dyn_eta = 1.0
dyn_beta = 1.0/2

# obstacle
center_e=np.array([0.0,1.0,0.25])
axes_e=np.array([a,b,c])
v_e = np.array([0.0,0.2,0.0])

#agent
center_agent1=np.array(MP.x_0)
center_agent2=np.array(MP2.x_0)

axes_agent=np.array([0.1,0.1,0.1])

v_agent1=np.array([0.0,0.0,0.0])
v_agent2=np.array([0.0,0.0,0.0])


obst_dynamic = sq_dyn(center=center_e, axis=axes_e, coeffs=np.array([1,1,2]), lmbda=dyn_lambda, beta=dyn_beta, eta=dyn_eta)
agent1_dynamic= sq_dyn(center=center_agent2, axis=axes_agent, coeffs=np.array([1,1,1]), lmbda=dyn_lambda, beta=dyn_beta, eta=dyn_eta)
agent2_dynamic= sq_dyn(center=center_agent1, axis=axes_agent, coeffs=np.array([1,1,1]), lmbda=dyn_lambda, beta=dyn_beta, eta=dyn_eta)




In [ ]:
def perturb_dynamic_agent1(x, v):
    out = obst_dynamic.gen_external_force(x, v-v_e)+\
    agent1_dynamic.gen_external_force(x,v-v_agent2)
    # print(out)
    return out

In [ ]:
def perturb_dynamic_agent2(x, v):
    out = obst_dynamic.gen_external_force(x, v-v_e)+\
    agent2_dynamic.gen_external_force(x,v-v_agent1)
    return out

In [ ]:
#DMP 3D plot
def plotresult():
    fig=plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(gamma_[:, 0], gamma_[:, 1], gamma_[:, 2],'--k', linewidth=1.5)
    ax.plot(gamma2_[:, 0], gamma2_[:, 1], gamma2_[:, 2],'--k', linewidth=1.5)
    # ax.plot(gamma[:, 0], gamma[:, 1], gamma[:, 2],'-g', linewidth=1.5)
    ax.plot(gamma_tra[:, 0], gamma_tra[:, 1], gamma_tra[:, 2],'-r', linewidth=1.5)
    ax.plot(gamma2_tra[:, 0], gamma2_tra[:, 1], gamma2_tra[:, 2],'-r', linewidth=1.5)
    ax.plot_surface(x, y, z, color='b', alpha=0.3, edgecolor=(0, 0, 0, 0.1))  # Use alpha for transparency
    ax.plot_surface(x_, y_, z_, color='r', alpha=0.3, edgecolor=(0, 0, 0, 0.1))  # Use alpha for transparency
    
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    
    ax.view_init(elev=10, azim=20)
    
    # Manually adjust the subplot margins
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
    plt.draw()         # Redraw the current figure
    plt.pause(0.1) 
    buf = BytesIO()
    plt.savefig(buf, format="png")
    
    

In [ ]:
plt.ion()

flag_conv = False
flag_conv2 = False
total_time=0

while not (flag_conv and flag_conv2):
    obst_dynamic.center=obst_dynamic.center+v_e*dt
    total_time += dt
    
    close_flag= np.linalg.norm(MP.x - MP2.x)>0.2
    # close_flag=True

        # Parametric equations for the ellipsoid
    x_ = a * np.sin(phi) * np.cos(theta) + obst_dynamic.center[0]
    y_ = b * np.sin(phi) * np.sin(theta) + obst_dynamic.center[1]
    z_ = c * np.cos(phi) + obst_dynamic.center[2]

    if not close_flag:
        print("agent1 stop")

    if (not flag_conv) and close_flag:
        MP.step(external_force=perturb_dynamic_agent1)
        gamma_tra = np.append(gamma_tra, np.array([MP.x]), axis=0)
        flag_conv = np.linalg.norm(MP.x - MP.x_goal) < MP.tol
        agent2_dynamic.center=np.array(MP.x)
        v_agent1=np.array(MP.dx)
    

    if not flag_conv2:
        MP2.step(external_force=perturb_dynamic_agent2)
        gamma2_tra = np.append(gamma2_tra, np.array([MP2.x]), axis=0)
        flag_conv2 = np.linalg.norm(MP2.x - MP2.x_goal) < MP2.tol
        agent1_dynamic.center=np.array(MP2.x)
        v_agent2=np.array(MP2.dx)


    if flag_conv:
        print("agent1 achieve goal")

    if flag_conv2:
        print("agent2 achieve goal")
    
    plotresult()


print(total_time)
    

In [ ]:
gamma_[-1]

In [ ]:
gamma_tra[-1]

In [ ]:
def my_function():
    print(v_e)

In [ ]:
for i in range(5):
    my_function()
    v_e=v_e+1

In [ ]:
A=np.array([1,2])
B=np.array([A])

In [ ]:
print(A)
print(B)

In [ ]:
plotresult()

In [ ]:
print(flag_conv)
print(flag_conv2)

In [ ]:
gamma_tra[-1]

In [ ]:
if False:
    print("hello")

In [ ]:
A=np.array([1,2,3])
print(A.ndim)
A.reshape(1,-1)

In [ ]:
A

In [ ]:
A